In [1]:
import numpy as np
from scipy.spatial import ConvexHull
from numpy.linalg import lstsq, norm
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (needed for 3D projection)

In [2]:

# -------------------------
# Linear-algebra helpers
# -------------------------
def barycentric_coords(point, vertices):
    """
    Return barycentric coords beta for `point` wrt simplex `vertices`.
    vertices: (k, d) array where k = simplex size (d+1 for full simplex in d-dim).
    Returns beta (k,) such that sum beta = 1 and point ~= sum beta_i * v_i (least squares).
    """
    v0 = vertices[0]
    A = (vertices[1:] - v0).T  # d x (k-1)
    rhs = point - v0
    if A.size == 0:
        return np.array([1.0])
    c, *_ = lstsq(A, rhs, rcond=None)
    beta_rest = c
    beta0 = 1.0 - beta_rest.sum()
    beta = np.concatenate(([beta0], beta_rest))
    return beta

def point_in_simplex(point, vertices, tol=1e-9):
    beta = barycentric_coords(point, vertices)
    return np.all(beta >= -tol), beta




def project_to_affine(point, vertices):
    """
    Orthogonal projection of `point` onto affine hull of `vertices`.
    """
    v0 = vertices[0]
    A = (vertices[1:] - v0).T  # d x (k-1)
    if A.size == 0:
        return v0.copy()
    # Orthonormal basis for column space of A via SVD
    U, s, Vt = np.linalg.svd(A, full_matrices=False)
    coords = U.T @ (point - v0)
    proj = v0 + U @ coords
    return proj



In [3]:
point = np.array([2,1])
vertices = np.array([[1,1],[1,-1],[-1,1]])
print(barycentric_coords(point, vertices))
print(project_to_affine(point, vertices))


[ 1.5  0.  -0.5]
[2. 1.]


In [4]:

# -------------------------
# Helper: find facet containing (or nearest) boundary point
# -------------------------
def find_containing_facet(hull, points, x, tol=1e-8):
    """
    Return a facet's vertex indices that contain point x (within tol).
    If none contain x (numerical), return the facet whose affine projection is nearest to x.
    """
    best = None
    best_dist = np.inf
    for simplex in hull.simplices:
        verts = points[simplex]
        q = project_to_affine(x, verts)
        inside, beta = point_in_simplex(q, verts, tol=tol)
        if inside:
            return simplex, q, beta
        d = norm(x - q)
        if d < best_dist:
            best_dist = d
            best = (simplex, q, beta)
    return best  # returns nearest facet if none contains x exactly

In [5]:
# -------------------------
# Face-walking algorithm
# -------------------------
def face_walk_projection(S, p, x0, max_iters=1000, tol=1e-9, report_every=50):
    """
    Inputs:
      S : (m, d) array of generator points
      p : (d,) point to project onto conv(S)
      x0: (d,) starting point on (or near) boundary of conv(S)
    Returns:
      x_proj : approximated projection point
      path   : list of boundary points visited (including start and finishing point)
      info   : dictionary with extra information (iterations, final_face, reason)
    """
    S = np.asarray(S) # point cloud
    p = np.asarray(p) # external point
    x = np.asarray(x0).astype(float) # inital point 
    if S.ndim != 2:
        raise ValueError("S must be shape (m, d)")
    d = S.shape[1]

    hull = ConvexHull(S)

    # If x0 is not on boundary (numerical), snap it to nearest boundary point on hull:
    simplex_info = find_containing_facet(hull, S, x, tol=1e-8)
    if simplex_info is None:
        raise RuntimeError("Could not find any facet on hull (degenerate?).")
    simplex, q_proj, beta = simplex_info
    
    # If q_proj is not very close to x, project x onto hull facet affine hull and inside simplex
    if norm(x - q_proj) > 1e-8:
        # snap x to the boundary nearest point (affine projection then clamp to simplex)
        # if projection not inside, choose nearest vertex of simplex
        inside, beta_q = point_in_simplex(q_proj, S[simplex], tol=1e-9)
        if inside:
            x = q_proj
        else:
            # snap to nearest vertex of that facet
            vertices = S[simplex]
            dists = np.linalg.norm(vertices - x, axis=1)
            x = vertices[np.argmin(dists)]

    path = [x.copy()]
    reason = "max_iters"
    final_face = None

    for it in range(1, max_iters + 1):
        # Recompute hull (points didn't change) - hull stable
        simplex_info = find_containing_facet(hull, S, x, tol=1e-9)
        if simplex_info is None:
            reason = "no_facet_found"
            break
        simplex, q_aff, beta_x = simplex_info
        verts = S[simplex]  # shape (k, d)
        k = verts.shape[0]

        # Project p to affine hull of the face
        p_aff = project_to_affine(p, verts)
        inside, beta_aff = point_in_simplex(p_aff, verts, tol=1e-9)
        if inside:
            # p_aff is projection onto convex hull (foot lies inside face)
            x = p_aff
            path.append(x.copy())
            reason = "found_projection_on_face"
            final_face = simplex
            break

        # Compute tangent projected gradient g_tangent within aff(F)
        g = x - p  # gradient of 0.5*||x-p||^2
        # Represent tangent space by columns A = (v_i - v0) for i>0
        v0 = verts[0]
        A = (verts[1:] - v0).T  # d x (k-1)
        if A.size == 0:
            # Face is a single vertex - cannot move inside face. Check if it's projection:
            # if p-x is in normal cone: i.e., x is closest among vertices of hull.
            # We will test nearby vertices:
            dists = np.linalg.norm(S[hull.vertices] - p, axis=1)
            min_idx = hull.vertices[np.argmin(dists)]
            if np.allclose(S[min_idx], x, atol=1e-8):
                reason = "projection_is_vertex"
                final_face = (simplex,)
                break
            else:
                x = S[min_idx].copy()
                path.append(x.copy())
                continue

        # Project g onto column space of A (tangent space directions)
        # Use least squares to find c such that A @ c = g_proj (in column space)
        U, svals, Vt = np.linalg.svd(A, full_matrices=False)
        coeffs = U.T @ g
        g_tangent = U @ coeffs  # orthogonal projection onto column space(A)
        g_tangent_norm = norm(g_tangent)
        if g_tangent_norm <= tol:
            # stationary in relative interior of current face but foot not in face -> treat as converged
            reason = "stationary_in_face"
            final_face = simplex
            break

        # direction along boundary (within affine hull) that decreases distance
        d_dir = -g_tangent / g_tangent_norm

        # We need barycentric coords of current x: beta_x (already computed)
        # Express d_dir in barycentric derivative gamma:
        # Solve A * c = d_dir for c (A is d x (k-1)).
        c, *_ = lstsq(A, d_dir, rcond=None)
        # gamma_0 = -sum c; gamma_i = c_i for i>=1
        gamma = np.empty(k)
        gamma[0] = -c.sum()
        gamma[1:] = c

        # Compute possible positive steps s where some barycentric coord hits zero:
        s_candidates = []
        for j in range(k):
            gj = gamma[j]
            bj = beta_x[j]
            # we need s > 0 solving bj + s * gj = 0 => s = -bj / gj, only if gj < 0 and bj > 0 (moving reduces that barycentric)
            if gj < -1e-12:
                s = -bj / gj
                if s > 1e-15:
                    s_candidates.append(s)
        if len(s_candidates) == 0:
            # No boundary to hit in forward direction (numerical or degenerate).
            # take a small step and continue (safeguard)
            s_star = 1e-6
        else:
            s_star = min(s_candidates)

        # move and snap numeric eps
        x = x + s_star * d_dir
        path.append(x.copy())

        # report progress occasionally
        if (it % report_every) == 0 or it == 1:
            current_dist = norm(x - p)
            print(f"[iter {it}] dist={current_dist:.6e} s_star={s_star:.3e} face_size={k}")

    info = {"iterations": it, "reason": reason, "final_face": final_face}
    x_proj = x
    return x_proj, np.array(path), info

In [6]:
import numpy as np
from numpy.linalg import lstsq, norm
from scipy.spatial import ConvexHull

def face_walk_projection_debug(S, p, x0, max_iters=1000, tol=1e-9, report_every=50, debug_iters=5):
    S = np.asarray(S)
    p = np.asarray(p)
    x = np.asarray(x0).astype(float)
    hull = ConvexHull(S)

    from scipy.spatial import Delaunay

    def find_containing_facet(hull, S, x, tol=1e-8):
        for simplex in hull.simplices:
            verts = S[simplex]
            A = np.hstack([verts, np.ones((len(verts), 1))])
            b = np.append(x, 1)
            try:
                beta = np.linalg.lstsq(A.T, b, rcond=None)[0]
            except np.linalg.LinAlgError:
                continue
            if np.all(beta >= -tol):
                q_proj = verts.T @ beta
                return simplex, q_proj, beta
        return None

    def project_to_affine(p, verts):
        v0 = verts[0]
        A = (verts[1:] - v0).T
        c, *_ = lstsq(A, (p - v0), rcond=None)
        return v0 + A @ c

    def point_in_simplex(p, verts, tol=1e-9):
        v0 = verts[0]
        A = (verts[1:] - v0).T
        c, *_ = lstsq(A, (p - v0), rcond=None)
        beta = np.empty(len(verts))
        beta[0] = 1 - np.sum(c)
        beta[1:] = c
        inside = np.all(beta >= -tol)
        return inside, beta

    path = [x.copy()]
    reason = "max_iters"
    final_face = None

    for it in range(1, max_iters + 1):
        simplex_info = find_containing_facet(hull, S, x, tol=1e-9)
        if simplex_info is None:
            reason = "no_facet_found"
            break
        simplex, q_aff, beta_x = simplex_info
        verts = S[simplex]
        k = verts.shape[0]

        p_aff = project_to_affine(p, verts)
        inside, beta_aff = point_in_simplex(p_aff, verts, tol=1e-9)
        if inside:
            x = p_aff
            path.append(x.copy())
            reason = "found_projection_on_face"
            final_face = simplex
            break

        g = x - p  # gradient
        v0 = verts[0]
        A = (verts[1:] - v0).T
        if A.size == 0:
            break

        # Tangent projection of g
        U, svals, Vt = np.linalg.svd(A, full_matrices=False)
        coeffs = U.T @ g
        g_tangent = U @ coeffs
        g_tangent_norm = norm(g_tangent)
        d_dir = -g_tangent / (g_tangent_norm + 1e-15)

        c, *_ = lstsq(A, d_dir, rcond=None)
        gamma = np.empty(k)
        gamma[0] = -c.sum()
        gamma[1:] = c

        s_candidates = []
        for j in range(k):
            gj = gamma[j]
            bj = beta_x[j]
            if gj < -1e-12:
                s = -bj / gj
                if s > 1e-15:
                    s_candidates.append(s)
        s_star = min(s_candidates) if s_candidates else 1e-6

        if it <= debug_iters:
            print(f"\n--- Iteration {it} ---")
            print(f"x = {x}")
            print(f"∇f(x) = {g}")
            print(f"||∇f(x)|| = {norm(g):.5f}")
            print(f"g_tangent = {g_tangent}")
            print(f"||g_tangent|| = {g_tangent_norm:.5f}")
            print(f"d_dir (unit) = {d_dir}")
            print(f"s_star = {s_star:.3e}")

        # x = x + s_star * d_dir
        # path.append(x.copy())
        
        # ---- safer barycentric update ----
        # beta_x: barycentric coords of current x (returned earlier by find_containing_facet)
        beta_new = beta_x + s_star * gamma

        # numerical fixes: clamp slight negatives, renormalize to sum to 1
        beta_new = np.maximum(beta_new, 0.0)
        if beta_new.sum() <= 0:
            # fallback (shouldn't happen) - pick nearest vertex
            idx_min = np.argmin(np.linalg.norm(verts - x, axis=1))
            x = verts[idx_min].copy()
            beta_new = np.zeros_like(beta_new); beta_new[idx_min] = 1.0
        else:
            beta_new = beta_new / beta_new.sum()
            # reconstruct Euclidean point from barycentrics
            x = (verts.T @ beta_new).copy()

        path.append(x.copy())
        # update beta_x for next iteration
        beta_x = beta_new


        if it <= debug_iters:
            print(f"x_next = {x}")

        if (it % report_every) == 0:
            print(f"[iter {it}] dist={norm(x - p):.6e} s_star={s_star:.3e} face_size={k}")

    info = {"iterations": it, "reason": reason, "final_face": final_face}
    return x, np.array(path), info


In [7]:
# # -------------------------
# # Visualization helpers
# # -------------------------
# def plot_path_2d(S, hull, path, p=None, x0=None, figsize=(7,7)):
#     plt.figure(figsize=figsize)
#     # plot hull polygon (edges)
#     for simplex in hull.simplices:
#         a, b = hull.points[simplex]
#         plt.plot([a[0], b[0]], [a[1], b[1]], '-k', lw=1)
#     # plot points
#     plt.scatter(S[:,0], S[:,1], s=10, alpha=0.5)
#     # path
#     plt.plot(path[:,0], path[:,1], '-o', lw=2, markersize=4, label='face-walk path')
#     if p is not None:
#         plt.scatter([p[0]], [p[1]], c='r', s=50, marker='x', label='p (to project)')
#     if x0 is not None:
#         plt.scatter([x0[0]], [x0[1]], c='g', s=50, marker='*', label='x0 (start)')
#     plt.gca().set_aspect('equal', adjustable='box')
#     plt.legend()
#     plt.title("Face-walking path on convex hull (2D)")
#     plt.show()

# def plot_path_3d(S, hull, path, p=None, x0=None, figsize=(9,7)):
#     fig = plt.figure(figsize=figsize)
#     ax = fig.add_subplot(111, projection='3d')
#     # plot triangular facets as wireframe
#     for simplex in hull.simplices:
#         tri = S[simplex]
#         # close triangle
#         tri = np.vstack((tri, tri[0]))
#         ax.plot(tri[:,0], tri[:,1], tri[:,2], color='k', linewidth=0.7)
#     # scatter points
#     ax.scatter(S[:,0], S[:,1], S[:,2], s=8, alpha=0.4)
#     # path
#     ax.plot(path[:,0], path[:,1], path[:,2], '-o', lw=2, markersize=3, color='C1', label='face-walk path')
#     if p is not None:
#         ax.scatter([p[0]], [p[1]], [p[2]], c='r', s=60, marker='x', label='p (to project)')
#     if x0 is not None:
#         ax.scatter([x0[0]], [x0[1]], [x0[2]], c='g', s=60, marker='*', label='x0 (start)')
#     ax.set_title("Face-walking path on convex hull (3D)")
#     plt.legend()
#     plt.show()

In [8]:
import plotly.graph_objects as go

def plot_path_2d(S, hull, path, p=None, x0=None):
    """
    Interactive 2D plot of face-walking path on convex hull using Plotly.
    """
    fig = go.Figure()

    # --- Convex hull edges ---
    for simplex in hull.simplices:
        a, b = hull.points[simplex]
        fig.add_trace(go.Scatter(
            x=[a[0], b[0]], y=[a[1], b[1]],
            mode="lines",
            line=dict(color="black", width=1),
            showlegend=False,
            hoverinfo="skip"
        ))

    # --- All points in S ---
    fig.add_trace(go.Scatter(
        x=S[:, 0], y=S[:, 1],
        mode="markers",
        marker=dict(size=6, color="gray", opacity=0.5),
        name="S (generator points)"
    ))

    # --- Path on boundary ---
    fig.add_trace(go.Scatter(
        x=path[:, 0], y=path[:, 1],
        mode="lines+markers",
        line=dict(color="orange", width=3),
        marker=dict(size=6, color="orange"),
        name="Face-walk path"
    ))

    # --- Outside point ---
    if p is not None:
        fig.add_trace(go.Scatter(
            x=[p[0]], y=[p[1]],
            mode="markers+text",
            marker=dict(symbol="x", color="red", size=10),
            text=["p (outside point)"],
            textposition="top right",
            name="p (to project)"
        ))

    # --- Starting point ---
    if x0 is not None:
        fig.add_trace(go.Scatter(
            x=[x0[0]], y=[x0[1]],
            mode="markers+text",
            marker=dict(symbol="star", color="green", size=12),
            text=["x₀ (start)"],
            textposition="bottom left",
            name="x₀ (start)"
        ))

    fig.update_layout(
        title="Face-Walking Path on Convex Hull (2D)",
        xaxis=dict(scaleanchor="y", scaleratio=1, title="x"),
        yaxis=dict(title="y"),
        width=700, height=700,
        legend=dict(x=0.02, y=0.98)
    )

    fig.show()


In [9]:
import plotly.graph_objects as go
import numpy as np

def plot_path_3d(S, hull, path, p=None, x0=None):
    fig = go.Figure()

    # --- Plot convex hull faces ---
    for simplex in hull.simplices:
        pts = S[simplex]
        tri = go.Mesh3d(
            x=pts[:, 0], y=pts[:, 1], z=pts[:, 2],
            i=[0], j=[1], k=[2],
            opacity=0.3, color="lightblue", name="Convex Hull"
        )
        fig.add_trace(tri)

    # --- Plot path taken ---
    if len(path) > 1:
        path = np.array(path)
        fig.add_trace(go.Scatter3d(
            x=path[:, 0], y=path[:, 1], z=path[:, 2],
            mode="lines+markers",
            line=dict(color="red", width=5),
            marker=dict(size=3, color="red"),
            name="Path"
        ))

    # --- Plot outside point ---
    if p is not None:
        fig.add_trace(go.Scatter3d(
            x=[p[0]], y=[p[1]], z=[p[2]],
            mode="markers+text",
            marker=dict(symbol="diamond", color="black", size=6),
            text=["Outside point p"],
            textposition="bottom center",
            name="Outside point p"
        ))

    # --- Starting point x₀ ---
    if x0 is not None:
        fig.add_trace(go.Scatter3d(
            x=[x0[0]], y=[x0[1]], z=[x0[2]],
            mode="markers+text",
            marker=dict(symbol="circle", color="green", size=6),
            text=["x₀ (start)"],
            textposition="bottom center",
            name="x₀ (start)"
        ))

    fig.update_layout(
        title="Face-Walking Path on Convex Hull (3D)",
        scene=dict(
            xaxis_title="X",
            yaxis_title="Y",
            zaxis_title="Z",
            aspectmode="data"
        ),
        legend=dict(x=0.02, y=0.98)
    )

    fig.show()


In [ ]:


# -------------------------
# Example usage
# -------------------------
if __name__ == "__main__":
    # Example 2D polygon
    # create random points on a circle plus interior points
    np.random.seed(1)
    angles = np.linspace(0, 2*np.pi, 30, endpoint=False)
    circle_pts = np.column_stack([np.cos(angles), np.sin(angles)]) * 1.0
    extra = 0.2 * (np.random.randn(20,2))
    S2 = np.vstack([circle_pts, extra])
    hull2 = ConvexHull(S2)
    # choose outside point p
    p2 = np.array([2.0, 0.3])
    # choose a start boundary point x0 (take a hull vertex)
    x0_2 = S2[hull2.vertices[5]]

    proj2, path2, info2 = face_walk_projection_debug(S2, p2, x0_2, max_iters=1000000, report_every=50)
    print("2D proj:", proj2, "info:", info2)
    plot_path_2d(S2, hull2, path2, p=p2, x0=x0_2)

    # # Example 3D: hemisphere + apex (ice-cream)
    # def sample_hemisphere(radius=1.0, n_theta=24, n_phi=64):
    #     pts = []
    #     thetas = np.linspace(0, np.pi/2, n_theta)
    #     phis = np.linspace(0, 2*np.pi, n_phi, endpoint=False)
    #     for th in thetas:
    #         for ph in phis:
    #             x = radius * np.sin(th) * np.cos(ph)
    #             y = radius * np.sin(th) * np.sin(ph)
    #             z = radius * np.cos(th)
    #             if z >= 0:
    #                 pts.append([x,y,z])
    #     return np.array(pts)

    # hemi = sample_hemisphere(1.0, n_theta=18, n_phi=72)
    # apex = np.array([[0.0, 0.0, 2.0]])
    # S3 = np.vstack([hemi, apex])
    # hull3 = ConvexHull(S3)
    # p3 = np.array([0.4, -0.2, 3.2])
    # # choose start boundary point x0 as some hull vertex (take one from hemisphere rim)
    # x0_3 = S3[hull3.vertices[10]]

    # proj3, path3, info3 = face_walk_projection_debug(S3, p3, x0_3, max_iters=10000, report_every=100)
    # print("3D proj:", proj3, "info:", info3)
    # plot_path_3d(S3, hull3, path3, p=p3, x0=x0_3)



--- Iteration 1 ---
x = [0.66913061 0.74314483]
∇f(x) = [-1.33086939  0.44314483]
||∇f(x)|| = 1.40271
g_tangent = [-0.81623645  0.90652241]
||g_tangent|| = 1.21985
d_dir (unit) = [ 0.66913061 -0.74314483]
s_star = 2.091e-01
x_next = [0.80901699 0.58778525]

--- Iteration 2 ---
x = [0.80901699 0.58778525]
∇f(x) = [-1.19098301  0.28778525]
||∇f(x)|| = 1.22526
g_tangent = [-0.42236042  0.73154971]
||g_tangent|| = 0.84472
d_dir (unit) = [ 0.5       -0.8660254]
s_star = 2.091e-01
x_next = [0.91354546 0.40673664]

--- Iteration 3 ---
x = [0.91354546 0.40673664]
∇f(x) = [-1.08645454  0.10673664]
||∇f(x)|| = 1.09169
g_tangent = [-0.31783196  0.5505011 ]
||g_tangent|| = 0.63566
d_dir (unit) = [ 0.5       -0.8660254]
s_star = 1.000e-06
x_next = [0.91354546 0.40673664]

--- Iteration 4 ---
x = [0.91354546 0.40673664]
∇f(x) = [-1.08645454  0.10673664]
||∇f(x)|| = 1.09169
g_tangent = [-0.31783196  0.5505011 ]
||g_tangent|| = 0.63566
d_dir (unit) = [ 0.5       -0.8660254]
s_star = 1.000e-06
x_next 


--- Iteration 1 ---
x = [0.54650904 0.65130411 0.52643216]
∇f(x) = [ 0.14650904  0.85130411 -2.67356784]
||∇f(x)|| = 2.80965
g_tangent = [ 0.55953727  1.30204533 -2.32116243]
||g_tangent|| = 2.71960
d_dir (unit) = [-0.20574284 -0.47876436  0.8534955 ]
s_star = 3.838e-01
x_next = [0.46753785 0.46753785 0.85403316]

--- Iteration 2 ---
x = [0.46753785 0.46753785 0.85403316]
∇f(x) = [ 0.06753785  0.66753785 -2.34596684]
||∇f(x)|| = 2.44003
g_tangent = [ 0.53876162  1.09933488 -1.97754745]
||g_tangent|| = 2.32583
d_dir (unit) = [-0.23164252 -0.47266304  0.85025373]
s_star = 2.703e-01
x_next = [0.40492095 0.33976902 1.08387116]

--- Iteration 3 ---
x = [0.40492095 0.33976902 1.08387116]
∇f(x) = [ 0.00492095  0.53976902 -2.11612884]
||∇f(x)|| = 2.18389
g_tangent = [ 0.47614472  0.97156605 -1.74770945]
||g_tangent|| = 2.05552
d_dir (unit) = [-0.23164252 -0.47266304  0.85025373]
s_star = 1.000e-06
x_next = [0.40492128 0.3397693  1.0838704 ]

--- Iteration 4 ---
x = [0.40492128 0.3397693  1.08